In [2]:
# get the rapos using gala and APW's new potential

# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

# Gala
import gala.dynamics as gd
import gala.potential as gp

In [3]:
# LOAD THE DATA
path = '/Users/dhortadarrington/Documents/Master/data/dr17_dr3_McMillan_astroNN_rev1.fits'
tb = fits.open(path)
data = tb[1].data 

print(len(data))

733900


In [4]:
mask = np.load('../sav/mask_parent.npy', allow_pickle=True)
mask_une = np.load('../sav/mask_unevolved.npy', allow_pickle=True)
print(len(data[mask]), len(data[mask&mask_une]))

194257 4531


In [5]:
coord.galactocentric_frame_defaults.set("v4.0")
potential = gp.MilkyWayPotential2022()

In [6]:
icrs = coord.SkyCoord(
    ra=data['RA'][mask&mask_une]*u.degree,
    dec=data['DEC'][mask&mask_une]*u.degree,
    distance=data['weighted_dist'][mask&mask_une]*u.pc,
    pm_ra_cosdec=data['pmra'][mask&mask_une]* u.mas / u.yr,
    pm_dec=data['pmdec'][mask&mask_une] * u.mas / u.yr,
    radial_velocity=data['VHELIO_AVG'][mask&mask_une] * u.km / u.s,
)

icrs_err = coord.SkyCoord(
    ra=0 * u.deg,
    dec=0 * u.deg,
    distance=data['weighted_dist_error'][mask&mask_une] * u.pc,
    pm_ra_cosdec=data['pmra_error'][mask&mask_une] * u.mas / u.yr,
    pm_dec=data['pmdec_error'][mask&mask_une] * u.mas / u.yr,
    radial_velocity=data['VSCATTER'][mask&mask_une] * u.km / u.s,
)


In [7]:
galcen_frame = coord.Galactocentric()
galcen_frame
galcen = icrs.transform_to(galcen_frame)

In [8]:
galcen.velocity

<CartesianDifferential (d_x, d_y, d_z) in km / s
    [(-100.76898062,  46.97190042, -40.76258035),
     (-215.06525914, -37.15095514, -44.27580832),
     (-135.09172588, -17.59141553,  16.27045363), ...,
     ( 144.82953823, -20.58761211, 175.79544541),
     (   0.42564992,  33.12302176, -64.95558503),
     (   0.86988818,  73.26048616,  59.36251645)]>

In [9]:
w0 = gd.PhaseSpacePosition(galcen.data)
orbit = potential.integrate_orbit(w0, dt=-0.5 * u.Myr, n_steps=10000)


In [ ]:
orbit.

In [10]:
# rads = np.sqrt(Rgal**2+Zgal**2)
rapo = np.array(orbit.apocenter())
rperi = np.array(orbit.pericenter())
ecc = np.array(orbit.eccentricity())
zmax = np.array(orbit.zmax())
# actions = np.array(orbit.actions())
# frequencies = np.array(orbit.frequencies())
# angles = np.array(orbit.angles())


/Users/dhortadarrington/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhortadarrington/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
np.save('../sav/rapo_gala', rapo)
np.save('../sav/rperi_gala', rperi)
np.save('../sav/zmax_gala', zmax)
np.save('../sav/ecc_gala', ecc)
# np.save('../sav/actions_gala', actions)
# np.save('../sav/frequencies_gala', np.array(orbit.eccentricity()))
# np.save('../sav/angles_gala', np.array(orbit.eccentricity()))
